# SFT
在 SFT 阶段，我们不再使用全量的数据，而是会局限在我们最终的任务上——根据作者、形式生成一个“以假乱真”的诗词。并且让模型学会如何根据元数据，生成诗词。为了满足这个目标，训练过程会有如下修改：
1. 训练的数据会聚焦在我们选定的作者和形式对应的数据，而非全量数据。
2. 我们会随机删除一些元数据作为训练数据，来让模型学会在各种情况下生成诗词。
3. Loss 的计算方式也会有所改变，我们只计算模型生成的诗词部分的 Loss，忽略前面的元数据以及后续的 Padding 数据。

Notebook 中的训练代码是简化过的，方便快速运行和理解。

In [1]:
import random

import torch

from nanopoet.common import CharTokenizer
from nanopoet.dataset import load_raw_data, split_data
from nanopoet.model import GPTLanguageModel

# 初始化随机种子，让重复执行的结果稳定
random.seed(12345)
torch.manual_seed(12345)

# 首先加载数据、设备信息
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
data = load_raw_data("../raw")

# 初始化分词器
tokenizer = CharTokenizer("".join(["".join(list(d.values())) for d in data]))


In [2]:
# 初始化模型结构
block_size = 256
model = GPTLanguageModel(
    vocab_size=tokenizer.vocab_size,
    emb_size=256,
    block_size=block_size,
    layer_num=8,
    head_num=8,
    dropout=0.1,
).to(device)

# 加载 Pre Train 的训练结果
pretrain_state = torch.load("./output/03_mid_train_model.pt", map_location=device)
model.load_state_dict(pretrain_state, strict=True)

<All keys matched successfully>

In [3]:
from nanopoet.common import encode_poem, filter_poem, update_poem_author

# 构建 SFT 阶段使用的数据集
filtered_data = [d for d in data if filter_poem(d)]
updated_data = [update_poem_author(d) for d in filtered_data]
train, val = split_data(updated_data)
print("train:", len(train))
print("val:", len(val))

# 构建一个函数，来随机抹去一些数据的元数据
def erase_metadata(poem):
    new_poem = {
        "content": poem["content"],
    }
    if random.random() < 0.5:
        new_poem["author"] = poem["author"]
    if random.random() < 0.5:
        new_poem["title"] = poem["title"]
    if random.random() < 0.5:
        new_poem["style"] = poem["style"]
    return new_poem


# 随机擦除一些元数据
train_erased = [erase_metadata(p) for p in train]
val_erased = [erase_metadata(p) for p in val]

# 使用特殊 token 将每一首诗词转化成特定的序列
train_ps = [encode_poem(t) for t in train_erased]
val_ps = [encode_poem(t) for t in val_erased]

# 打印一些例子
print("转换后：", train_ps[0])

# 将转换后的序列编码成数据
train_data = [torch.tensor(tokenizer.encode(poem), dtype=torch.long) for poem in train_ps]
val_data = [torch.tensor(tokenizer.encode(poem), dtype=torch.long) for poem in train_ps]
# 提取一下训练数据里的 author 和 style，用于后续生成样本提示词
train_authors = list(set([p["author"] for p in train if "author" in p]))
train_styles = list(set([p["style"] for p in train if "style" in p]))


# 构建一个读取 sft 训练数据的函数
def get_batch(poems_encoded, batch_size, start_token_id, end_token_id, pad_token_id, block_size, device):
    """随机采样一批诗词（SFT版本，使用-1 mask）"""
    indices = torch.randint(len(poems_encoded), (batch_size,))
    batch = [poems_encoded[i] for i in indices]
    # 截断过长的数据
    batch = [p[:block_size] if len(p) > block_size else p for p in batch]
    max_len = max(len(p) for p in batch)
    batch_x = []
    batch_y = []
    for poem in batch:
        if len(poem) > 1:
            x = poem[:-1]  # 输入序列
            y = poem[1:]  # 目标序列

            # 创建 mask：找到 START_TOKEN 的位置
            # 注意：在 y 中查找，因为 y 是我们要预测的目标
            y_list = y.tolist()

            # 初始化为 -1（全部不计算 loss）
            y_masked = torch.full_like(y, -1)

            # 找到 START_TOKEN 在 y 中的位置
            try:
                start_idx = y_list.index(start_token_id)
                # 找到 END_TOKEN 在 y 中的位置
                end_idx = y_list.index(end_token_id, start_idx)
                # 从 START_TOKEN 到 END_TOKEN（包括两端）设置为真实 token
                y_masked[start_idx:end_idx + 1] = y[start_idx:end_idx + 1]
            except ValueError:
                # 如果没找到 START_TOKEN 或 END_TOKEN，整个序列都不计算 loss
                pass

            # 填充
            pad_len = max_len - 1 - len(x)
            if pad_len > 0:
                x = torch.cat([x, torch.full((pad_len,), pad_token_id, dtype=torch.long)])
                y_masked = torch.cat([y_masked, torch.full((pad_len,), -1, dtype=torch.long)])
            batch_x.append(x)
            batch_y.append(y_masked)

    return torch.stack(batch_x).to(device), torch.stack(batch_y).to(device)

train: 22084
val: 2454
转换后： BA欧阳修aT一斛珠tC今朝祖宴。可怜明夜孤灯馆。酒醒明月空床满。翠被重重，不似香肌暖。愁肠恰似沈香篆。千回万转萦还断。梦中若得相寻见。却愿春宵，一夜如年远。c


In [4]:
import torch.nn.functional as F
from nanopoet.common import CONTENT_START, CONTENT_END, PADDING, encode_poem_prompt

# 训练配置（SFT专用）
batch_size = 16  # SFT数据量较小，用小batch
learning_rate = 3e-4  # 基础学习率
init_lr_frac = 0.02  # 从mid的2%开始（非常保守，参考nanochat）
# 学习率调整参考 nanochat，从训练开始就做线性衰减

# 梯度裁剪
grad_clip = 1.0

epochs = 3  # SFT通常不需要太多轮
eval_interval = 10
eval_iters = 5
# 初始优化器
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate * init_lr_frac)

# Notebook 只训练少数步，演示为主
total_steps = 50

pad_token_id = tokenizer.encode(PADDING)[0]
start_token_id = tokenizer.encode(CONTENT_START)[0]
end_token_id = tokenizer.encode(CONTENT_END)[0]

for step in range(total_steps):
    xb, yb = get_batch(train_data, batch_size, start_token_id, end_token_id, pad_token_id, block_size, device)

    logits, _ = model(xb)

    # 计算loss（ignore_index=-1 会自动忽略条件和padding部分）
    loss = F.cross_entropy(
        logits.view(-1, tokenizer.vocab_size),
        yb.view(-1),
        ignore_index=-1
    )

    # 反向传播
    optimizer.zero_grad()
    loss.backward()

    # 梯度裁剪
    if grad_clip > 0.0:
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

    # 应用学习率调度
    lr_mult = 1.0 - step / total_steps
    for param_group in optimizer.param_groups:
        param_group['lr'] = (learning_rate * init_lr_frac) * lr_mult

    optimizer.step()
    # 定期评估、打印Loss、学习率变化，并生成一段测试样本
    if step % eval_interval == 0:
        model.eval()
        out = {}
        for split, data in [('train', train_data), ('val', val_data)]:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(data, batch_size, start_token_id, start_token_id, pad_token_id, block_size, device)
                logits, _ = model(X, Y)
                losses[k] = F.cross_entropy(
                    logits.view(-1, tokenizer.vocab_size),
                    Y.view(-1),
                    ignore_index=-1
                )
            out[split] = losses.mean()
        current_lr = (learning_rate * init_lr_frac) * lr_mult
        print(
            f"Step {step:5d} | 训练Loss: {out['train']:.4f} | 验证Loss: {out['val']:.4f} | LR: {current_lr:.2e} (x{lr_mult:.3f})")

        # 生成样本时，从训练数据的作者和风格中随机取一个，作为生成的 prompt
        test_author = random.choice(train_authors)
        test_syle = random.choice(train_syles)
        # 生成多种 prompt 来看看实际使用时不同情况的样本
        test_prompts = [
            encode_poem_prompt(),
            encode_poem_prompt(author=test_author),
            encode_poem_prompt(style=test_syle),
            encode_poem_prompt(author=test_author, style=test_syle),
            encode_poem_prompt(author=test_author, style=test_syle, title="咏柳"),
        ]
        for prompt in test_prompts:
            context = torch.tensor([tokenizer.encode(prompt)], dtype=torch.long, device=device)
            # 生成时不再限制长度，而是让模型生成到结束符位置
            generated = model.generate(context, max_new_tokens=block_size, stop_token_ids=[end_token_id])
            sample = tokenizer.decode(generated[0].tolist())
            print("生成样本：", sample)
        model.train()

Step     0 | 训练Loss: 3.9004 | 验证Loss: 4.1709 | LR: 6.00e-06 (x1.000)
生成样本： B晦代耿亡aS七韻言襯瑞二名丁題惡tC子tC破頭新，白分吉春牖。名紫寒割噉一盬足。嚼日事樓業英悄當巧經元將是舞，怪。c
生成样本： BA陆游a杖福s迎sT地㳇和t判消t蒂廛未空物愁菡間鬭，占戈犀然幸雖洞。煙茫千，是清。直草依軍。丁，官柏真消，在簾，應看深魚幾吹江壺虔。c
生成样本： BS沁园春s芳普aS編累務良畀經tC征韻雲自歲縉疏，趨雲人謁春對自天星燈住束昂西向還人的賢夜仙勁值如梅，德似昡賎。当勝力呼創，白，點遊生。c
生成样本： BA陆游aS沁园春sT五不晚三題楞祕錦S一梅大望蘇律诗地兩掖河穏楊憩荒搜日喜行，蒼風風猶和毛當春高，雨說光山雲勾失別，樂。房問前復千驛梅儒云醲老，虛人佇愠重月是漢傳知人飛。c
生成样本： BA陆游aS沁园春sT咏柳tC霄，倚士還了寒短君合，得筍是人羣同。篿吳風止齊萊日斜。多伴二，霞蘚未允公道辟談來。樹宴，色輕築苦運來子，風樣籍稀韻界，春人腸得山。弭入鶴得他芳相郢。纚。c
Step    10 | 训练Loss: 3.2729 | 验证Loss: 3.8013 | LR: 4.80e-06 (x0.800)
生成样本： BA觉。七aS七言律诗sT長到道t三有詠中未二tC汙林青銀，今漢休。獨記得山焄朋秋得龍飛。西疏古，卜歲發師为带千一耐成在來間，迴舊，客關間殊便功。詩。嚼，，斗到衰。室，在寂。c
生成样本： BA陆游aS七言绝來诗sTt愴雲露丰眙落滄釋二藥庭命有十遇，龍。到萬得此過袍前，喜馳飛。c
生成样本： BS七言绝句s藉方此立德举鳴孔沙貯禁tC清之通筠醉還北遠儘霜，去重抵宮桑降奏故又良風夕根桂清背江，好期真看邊。c
生成样本： BA陆游aS七言绝句s惬送韵醉tC夜自詩忍郡紺憶知魄練，艷名愛漁草耿冠。不長柳城圩髯何須公符事，作江静爛聊酬相弄一吹鳥眼不走雲塞長落園愜塔，枕家野無志，久恨別趣芳老報忍日戶。c
生成样本： BA陆游aS七言绝句sT咏柳tC名，絲處天道兵怨繫江笑妨漿親力何瀟人傅陰長雲色想寐戶義山黄南桃，快正何。c
Step    20 | 训练Loss: 2.8307 | 验证Loss: 3.6378 | LR: 3.60e-06 (x0.600)
生成样本： BA籍出花aS七徐成

In [5]:
from pathlib import Path

# 把训练结果保存下来，给后续训练使用
output_path = Path("./output/04_sft_model.pt")
output_path.parent.mkdir(parents=True, exist_ok=True)
torch.save(model.state_dict(), output_path)
print(f"模型已保存到 {output_path}")

模型已保存到 output/04_sft_model.pt
